# Assignment 4. Quantization for LLM


## Goals

본 실습에서는 대형 언어 모델(Large Language Model, LLM)에 대해 양자화(Quantization)을 수행하여 모델을 압축하는 방법을 실습합니다.

LLM은 파라미터의 개수가 매우 많기 때문에 일반적으로 FP16으로 관리합니다. 그럼에도 파라미터의 크기가 많이 크며, LLaMA-2 7B와 같이 작은 모델에 대해서도 모바일 환경에서 수행하고자 하는 경우 FP16에서도 최소 14GB 이상의 메모리를 요구하며 이는 실제로 돌리기에 무리가 있습니다. 따라서, 양자화를 통해 모델의 weight를 더 낮은 precision으로 압축하는 것이 가능합니다.

## Contents
1. Weight-only Quantization
  - Weight만 quantization을 적용합니다.
  - 장점: 매우 낮은 bit-width로 weight를 양자화할 수 있으며, 단일 배치 추론 환경에서 유리합니다.
  - 단점: Dequantization 후 FP16 연산을 수행해야 합니다.
  - 예시: AWQ (W3A16, W4A16)
2. Weight and Activation Quantization
  - Weight와 activation 모두 quantization을 적용합니다.
  - 장점: 낮은 precision의 연산을 통해 가속 가능하며, 대규모 배치 추론 환경에서 유리합니다.
  - 단점: weight의 bit-width를 낮추기에 한계가 존재합니다.
  - 예시: SmoothQuant (W8A8)

## Setup

실습에 필요한 패키지를 설치합니다.

In [1]:
print('Installing packages...')
# !pip install torch transformers==4.31.0 accelerate==0.21.0 sentencepiece==0.1.99 tokenizers==0.13.3 datasets==2.15.0 tqdm zstandard huggingface-hub==0.27.0
!curl -L https://huggingface.co/datasets/mit-han-lab/pile-val-backup/resolve/main/val.jsonl.zst -o "D:\\data\\val.jsonl.zst"
datapath = "D:\\data\\val.jsonl.zst"
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HOME"] = "D:\\data\\hf_cache"

Installing packages...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  1311  100  1311    0     0   1900      0 --:--:-- --:--:-- --:--:--  1905
100  1311  100  1311    0     0   1900      0 --:--:-- --:--:-- --:--:--  1902

 10  449M   10 45.6M    0     0  31.6M      0  0:00:14  0:00:01  0:00:13 31.6M
 41  449M   41  187M    0     0  77.2M      0  0:00:05  0:00:02  0:00:03  144M
 73  449M   73  328M    0     0  94.7M      0  0:00:04  0:00:03  0:00:01  139M
100  449M  100  449M    0     0   106M      0  0:00:04  0:00:04 --:--:--  146M


실습에 필요한 모듈을 로드합니다.

In [2]:
from tqdm import tqdm
import torch
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from functools import partial
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

다음 코드는 모델 크기를 계산하는 데 사용됩니다.

In [3]:
class LLMModel:
    def __init__(self, model_name):
        self.model_name = model_name
        self.model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16, use_safetensors=True)
        self.model.eval()

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        testenc = load_dataset('wikitext', 'wikitext-2-raw-v1', split='test')
        testenc = self.tokenizer("\n\n".join(testenc['text']), return_tensors='pt')
        self.testenc = testenc.input_ids.to(self.model.device)

        self.model_changed = False

    def _evaluate(self):
        nsamples = 10
        nlls = []
        for i in tqdm(range(nsamples), desc="evaluating..."):
            batch = self.testenc[:, (i * 2048):((i + 1) * 2048)].to(self.model.device)
            with torch.no_grad():
                lm_logits = self.model(batch).logits
            shift_logits = lm_logits[:, :-1, :].contiguous().float()
            shift_labels = self.testenc[:, (i * 2048):((i + 1) * 2048)][:, 1:]
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            neg_log_likelihood = loss.float() * 2048
            nlls.append(neg_log_likelihood)

        return torch.exp(torch.stack(nlls).sum() / (nsamples * 2048))

    def get_model_size(self, data_width=16, group_size=-1):
        if group_size != -1:
            data_width += (16 + 4) / group_size

        num_elements = 0
        for param in self.model.parameters():
            num_elements += param.numel()
        return num_elements * data_width

    def model_delete(self):
        del self.model
        gc.collect()
        torch.cuda.empty_cache()

    def model_evaluate(self, data_width, group_size):
        model_perplexity = self._evaluate()
        model_size = self.get_model_size(data_width=data_width, group_size=group_size)
        print(f"\nmodel perplexity: {model_perplexity:.2f}")
        print(f"model size: {model_size/1024/1024/8:.2f} MiB")
        return model_perplexity

    def model_reset(self):
        if self.model_changed:
            self.model_delete()
            self.model = AutoModelForCausalLM.from_pretrained(self.model_name, device_map="auto", torch_dtype=torch.float16, use_safetensors=True)
            self.model.eval()
            self.model_changed = False

    def model_change(self, model: nn.Module):
        self.model_delete()
        self.model = model
        self.model.eval()
        self.model_changed = True

먼저 FP32 모델의 혼란도(perflexity)와 모델 크기를 평가해봅시다.

LLaMA-65B 모델의 디코딩 단계에서 단일 배치 추론을 수행할 때, 우리는 $[1, 8192] \times [8192, 8192]$ 형태의 GEMV(General Matrix-Vector Multiplication)연산을 수행해야 합니다.

NVIDIA A100 80G의 경우, **half-precision(FP16)** 에서의 성능은 312TFLOPS이며, memory bandwidth는 약 2000GB/s 입니다. 이를 바탕으로, **계산 집약도(computation intensity)** 를 계산할 수 있습니다:

$$
\frac{\text{FLOP}}{\text{Byte}} = \frac{2\times 8192^2}{8192^2} << \frac{3.12\times 10^{11}}{2\times 10^9}
$$

이는 매우 메모리 제약적(Memory-bounded)(~$10^2$ gap)으로, 저비트 가중치 양자화가 필요한 이유입니다.

In [4]:
model_path = "facebook/opt-125m"
llm_model = LLMModel(model_path)

# Evaluate the model
llm_model.model_evaluate(data_width=32, group_size=128)

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

c:\ProgramData\miniconda3\envs\env_aias_test\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\data\hf_cache\hub\models--facebook--opt-125m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

evaluating...: 100%|██████████| 10/10 [00:01<00:00,  7.99it/s]



model perplexity: 28.67
model size: 480.08 MiB


tensor(28.6723, device='cuda:0')

# 4.1. Weight-only Quantization (AWQ)

AWQ (activation aware weight only quantization)

대형 언어 모델(LLM)은 다양한 작업에서 뛰어난 성능을 보여주고 있지만, 엄청난 모델 크기로 인해 하드웨어적 장벽(메모리 크기)이 높아지고, 토큰 생성 속도가 느려집니다(메모리 대역폭). LLM의 크기와 계산량은 기하급수적으로 증가하고 있는 반면, 메모리 대역폭은 느리게 증가하고 있습니다. 이 격차는 LLM 성능에서 중요한 병목 현상입니다. 이번 실습에서는 **새로운 양자화 알고리즘(AWQ)**을 사용하여 LLM의 메모리 사용량을 줄이고 추론 속도를 가속화하는 방법을 탐구할 것입니다.

## AWQ


Uniform quantization 은 실수 값을 range $[\beta, \alpha]$에서 $[0, 2^{b} - 1]$로 매핑하는 것입니다.

Notation:

- Quantized Weight: $w_q$

- Scale factor: $s_q$

- Zero Point: $z$
\begin{equation}
s_q = \frac{\alpha - \beta}{2^{b} - 1} \tag{1},
\end{equation}

\begin{equation}
z = -\text{Round}(\beta * scale) \tag{2}
\end{equation}

\begin{equation}
w_q = \text{Clamp}(\text{Round}(\frac{w}{s_q}) + z) \tag{3},
\end{equation}



## Pseudo Quantization
아래 코드는 의사 양자화(pseudo quantization)을 위한 것입니다.


Pseudo Quantization는 모델의 가중치를 실제로 양자화하지 않고, 양자화의 영향을 시뮬레이션하기 위해 사용됩니다. (즉, 가장 가까운 양자화된 값으로 반올림한 다음, **다시 부동 소수점으로 복원(dequantizing)하는** 것입니다.)

In [5]:
# core quantization method (simulated quantization)
def pseudo_quantize_tensor(w, n_bit=4, q_group_size=-1):
    org_w_shape = w.shape
    if q_group_size > 0:
        assert org_w_shape[-1] % q_group_size == 0
        w = w.reshape(-1, q_group_size)

    assert w.dim() == 2

    # Calculate the maximum (\alpha) and minimum values (\beta) in the tensor.
    max_val = w.amax(dim=1, keepdim=True)
    assert max_val.dim() == 2 and max_val.size(0) == w.size(0) and max_val.size(1) == 1
    min_val = w.amin(dim=1, keepdim=True)
    assert min_val.dim() == 2 and min_val.size(0) == w.size(0) and min_val.size(1) == 1

    # Calculate the scale factor and zero point.  (Formula 1 & 2)
    max_int = 2 ** n_bit - 1
    scales = (max_val - min_val).clamp(min=1e-5) / max_int
    assert scales.shape == max_val.shape
    zeros = (-torch.round(min_val / scales)).clamp_(0, max_int)
    assert scales.shape == min_val.shape

    assert torch.isnan(scales).sum() == 0
    assert torch.isnan(w).sum() == 0

    # Quantize W: Map values in the range [\beta, \alpha] to lie within [0, 2^b - 1] (Formula 3)
    w = torch.clamp(torch.round(w / scales) + zeros, 0, max_int)
    assert w.dim() == 2 and w.size(0) == scales.size(0) and w.size(1) == q_group_size

    # Dequantize W (pseudo quantization, the inverse transformation of Formula 3)
    w = (w - zeros) * scales
    assert w.dim() == 2 and w.size(0) == scales.size(0) and w.size(1) == q_group_size

    assert torch.isnan(w).sum() == 0

    w = w.reshape(org_w_shape)
    return w

@torch.no_grad()
def pseudo_quantize_model_weight(
    model, w_bit, q_group_size,
):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear):
            m.weight.data = pseudo_quantize_tensor(m.weight.data, n_bit=w_bit, q_group_size=q_group_size)

이제 quantized 3-bit 모델의 혼란도(perplexity)와 크기를 평가해 봅시다.

In [6]:
llm_model.model_reset()
pseudo_quantize_model_weight(llm_model.model, w_bit=3, q_group_size=128)
llm_model.model_changed = True

# Evaluate the model
llm_model.model_evaluate(data_width=3, group_size=128)

evaluating...: 100%|██████████| 10/10 [00:00<00:00, 24.64it/s]


model perplexity: 64.88
model size: 47.12 MiB


tensor(64.8840, device='cuda:0')

모델 크기가 줄어든 것은 확인할 수 있지만, 혼란도(perplexity)는 상당히 증가했습니다.

논문에서의 관찰에 따르면 LLM의 활성화(activations)에서 일부 채널에 **아웃라이어(outliers)**가 소량 발생하고 있습니다. 특정 채널에 아웃라이어가 있는 경우, 이는 **모든 토큰에서 지속적으로 나타납니다.**

주어진 토큰에 대한 채널 간의 분산(variance)은 크지만(일부 채널의 활성화는 매우 크고, 대부분은 작습니다), 특정 채널의 크기(magnitude)가 토큰 간에 가지는 분산은 작습니다(아웃라이어 채널은 지속적으로 큽니다).


AWQ(Activation Aware Weight Quantization)의 기법에 따르면, 활성화(activation) 아웃라이어에 해당하는 가중치 채널은 더 두드러지며, 이러한 두드러진 가중치를 보존하는 것이 성능 향상으로 이어질 수 있습니다. 다음으로, 두드러진 가중치를 찾고 원래 값으로 유지하여 혼란도(perplexity)의 변화를 관찰해 보겠습니다.

아래 코드는 calibration 데이터셋을 로드하여 활성화 아웃라이어를 얻고 두드러진 가중치를 식별하는 데 사용됩니다.

In [7]:
def get_calib_dataset(tokenizer=None, n_samples=256, block_size=512):
    dataset = load_dataset("mit-han-lab/pile-val-backup", split="validation")
    dataset = dataset.shuffle(seed=42)
    samples = []
    n_run = 0
    for data in dataset:
        line = data["text"]
        line = line.strip()
        line_encoded = tokenizer.encode(line)
        if len(line_encoded) > block_size:
            continue
        sample = torch.tensor([line_encoded])
        if sample.numel() == 0:
            continue
        samples.append(sample)
        n_run += 1
        if n_run == n_samples:
            break

    # now concatenate all samples and split according to block size
    cat_samples = torch.cat(samples, dim=1)
    n_split = cat_samples.shape[1] // block_size
    print(f" * Split into {n_split} blocks")
    return [cat_samples[:, i*block_size:(i+1)*block_size] for i in range(n_split)]

@torch.no_grad()
def get_calib_feat(model, tokenizer):
    input_dict = dict()
    def stat_input_max_hook(m, x, y, name):
        if isinstance(x, tuple):
            x = x[0]
        x_max = x.view(-1, x.shape[-1]).abs().mean(dim=0).cpu().detach()
        if name not in input_dict:
            input_dict[name] = [x_max]
        else:
            input_dict[name] += [x_max]

    hooks = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Linear):
            hooks.append(
                m.register_forward_hook(
                    partial(stat_input_max_hook, name=name)))

    print("Collecting activation scales...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    samples = get_calib_dataset(tokenizer)
    pbar = tqdm(samples)
    for input_ids in pbar:
        input_ids = input_ids.to(device)
        model(input_ids)

    for hook in hooks:
        hook.remove()
    return input_dict

In [8]:
llm_model.model_reset()
input_feat = get_calib_feat(llm_model.model, llm_model.tokenizer)

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 4s [Retry 3/5].


README.md:   0%|          | 0.00/167 [00:00<?, ?B/s]

c:\ProgramData\miniconda3\envs\env_aias_test\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\data\hf_cache\hub\datasets--mit-han-lab--pile-val-backup. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Repo card metadata block was not found. Setting CardData to empty.


val.jsonl.zst:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/214670 [00:00<?, ? examples/s]

 * Split into 127 blocks


100%|██████████| 127/127 [00:04<00:00, 26.78it/s]


In [9]:
print(type(input_feat['model.decoder.layers.0.self_attn.q_proj']))
print(len(input_feat['model.decoder.layers.0.self_attn.q_proj']))
print(input_feat['model.decoder.layers.0.self_attn.q_proj'][0].shape)
print(sum(input_feat['model.decoder.layers.0.self_attn.q_proj']).shape)

<class 'list'>
127
torch.Size([768])
torch.Size([768])


# [실습 1] Scale 1% salient channels

1%의 가중치를 FP16으로 유지하면 모델 크기(총 비트 수로 측정)를 크게 늘리지 않고도 양자화 성능을 향상시킬 수 있지만, 이러한 혼합 정밀도 데이터 유형은 시스템 구현을 어렵게 만듭니다.

따라서 중요한 가중치를 실제로 FP16으로 유지하지 않고 중요한 가중치를 보호할 수 있는 방법을 찾아야 합니다.

AWQ의 방법론에 따르면, 중요한 가중치 채널을 단순히 스케일링하여(특정한 값을 곱해 주어) 보호할 수 있습니다. 원리는 다음과 같습니다:

- Linear layer channel $\mathbf{y} = \mathbf{w}x$ (from $\mathbf{W}x$)일 때, 우리가 주목해야 할 것은 양자화 함수 $Q(\mathbf{w})x$으로 발생하는 quantization error입니다.

- Quantization function $Q(\mathbf{w})$ = $Δ\cdot Round(\frac{\mathbf{w}}{Δ})$, $Δ = \frac{\max(|w|)}{2^{N - 1}}$.

- Quantization error $Err(Q(\mathbf{w}) x) = Δ\cdot RoundErr(\frac{\mathbf{w}}{Δ})\cdot x$
- 스케일링 된  Quantization error $Err(Q(\mathbf{w} \cdot s)(\frac{x}{s})) = Δ\cdot RoundErr(\frac{\mathbf{w}}{Δ})\cdot x\cdot \mathbf{\frac{1}{s}}$.
- $RoundErr$ 는 언제나 ~0.25 입니다 (0-0.5 사이의 평균이므로).
- 그룹의 크기가 충분히 클 때(e.g., 128), 하나의 채널을 스케일링하는 것은 일반적으로 그룹 내 최대 값을 증가시키지 않습니다 (즉, $Δ$ 는 변하지 않습니다).
- 그러므로, $Err(Q(\mathbf{w} \cdot s)(\frac{x}{s})) = Δ\cdot RoundErr(\frac{\mathbf{w}}{Δ})\cdot x\cdot \mathbf{\frac{1}{s}}$ < $Δ\cdot RoundErr(\frac{\mathbf{w}}{Δ})\cdot x = Err(Q(\mathbf{w}) x)$.

아래 코드를 완성하여 중요한 가중치 채널을 스케일링하고, 양자화 한 다음, 다시 스케일을 줄인 후 혼란도(perplexity)의 변화를 관찰해보세요.

In [10]:
@torch.no_grad()
def pseudo_quantize_model_weight_scaleup(
    model, w_bit, q_group_size, input_feat, scale_factor
):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear):
            importance = sum(input_feat[n]).float()
            # 1퍼센트 채널의 개수
            num_samples = int(len(importance) * 0.01)

            ############### YOUR CODE STARTS HERE ###############

            # Step 1: importance를 기준으로 1%의 중요한 채널을 찾으세요  (hint: use torch.topk())
            # hint : torch.topk() 함수를 사용하세요. torch.topk() 함수는 PyTorch에서 텐서의 값 중 상위 k개의 값과 그들의 인덱스를 반환하는 함수입니다. torch.topk()[0]는 값을, torch.topk()[1]은 인덱스를 반환합니다.
            outlier_mask = torch.topk(importance, k=num_samples)[1]

            ############### YOUR CODE ENDS HERE #################
            assert outlier_mask.dim() == 1

            # 스케일 팩터를 적용하는 것을 시뮬레이션하기 위해, 양자화 전에 스케일 팩터를 곱하고, 양자화 후에 스케일 팩터로 나눕니다.
            # scale_factor를 이용해 중요한 가중치 채널의 값을 확대합니다.
            m.weight.data[:, outlier_mask] *= scale_factor

            m.weight.data = pseudo_quantize_tensor(m.weight.data, n_bit=w_bit, q_group_size=q_group_size)

            ############### YOUR CODE STARTS HERE ###############

            # Step 2: scale_factor를 이용해 중요한 가중치 채널의 값을 다시 축소하세요.
            m.weight.data[:, outlier_mask] /= scale_factor

            ############### YOUR CODE ENDS HERE #################

In [23]:
llm_model.model_reset()
pseudo_quantize_model_weight_scaleup(llm_model.model, w_bit=3, q_group_size=128, input_feat=input_feat, scale_factor=2)
llm_model.model_changed = True

# Evaluate the model
llm_model.model_evaluate(data_width=3, group_size=128)

evaluating...: 100%|██████████| 10/10 [00:00<00:00, 25.95it/s]



model perplexity: 53.28
model size: 47.12 MiB


tensor(53.2786, device='cuda:0')

스케일링을 통해서 중요한 가중치를 보호함과 동시에, 모든 가중치를 3bit로 유지할 수 있었습니다.

In [ ]:
# model_path = "facebook/opt-125m"
# llm_model = LLMModel(model_path)

In [28]:
for scale_factor in [1,2,3,4]:
    llm_model.model_reset()
    pseudo_quantize_model_weight_scaleup(llm_model.model, w_bit=3, q_group_size=128, input_feat=input_feat, scale_factor=scale_factor)
    llm_model.model_changed = True

    # Evaluate the model
    print(f"scale_factor={scale_factor}")
    llm_model.model_evaluate(data_width=3, group_size=128)

scale_factor=1


evaluating...: 100%|██████████| 10/10 [00:00<00:00, 23.51it/s]



model perplexity: 64.88
model size: 47.12 MiB
scale_factor=2


evaluating...: 100%|██████████| 10/10 [00:00<00:00, 21.10it/s]



model perplexity: 53.28
model size: 47.12 MiB
scale_factor=3


evaluating...: 100%|██████████| 10/10 [00:00<00:00, 20.81it/s]



model perplexity: 69.95
model size: 47.12 MiB
scale_factor=4


evaluating...: 100%|██████████| 10/10 [00:00<00:00, 21.49it/s]



model perplexity: 154.36
model size: 47.12 MiB


코드에서 서로 다른 스케일링 팩터 $s$(예: 1, 2, 3, 4)를 시도하고 혼란도(perplexity)의 변화를 관찰해보세요.

혼란도(perplexity)가 먼저 감소하다가 다시 증가하는 것을 관찰했나요?

너무 큰 팩터로 스케일링하면 그룹 내 최대 값이 증가할 수 있습니다(즉,$Δ$가 증가함).

이는 다른 채널의 양자화에 영향을 미칠 수 있습니다.

# [실습 2] Scale factor search

지금까지 우리는 스케일링 팩터$s$를 직접 정의해 주었습니다.

그러나 Fine-tuning의 불안정성 때문에, 미리 정의된 검색 공간 내에서 최적의
$s$를 찾는 것이 더 나은 선택이 될 것입니다. 우리는 중요한 가중치를 보호하면서 다른 값을 고려하기 위해 검색 공간 내에서 최적의 스케일을 찾을 수 있습니다.

실제로, 논문에서는 활성화만 고려하는 것으로도 좋은 결과를 얻을 수 있음을 관찰할 수 있습니다.

우리는 스케일링 팩터 $s$를 활성화의 L1-norm (즉, acviation matrix의 절댓값들의 평균)의 $\alpha$제곱으로 설정할 것입니다.

$\alpha$의 값은 grid search를 통해 적절한 값으로 검색합니다.

검색을 위한 코드를 추가하고 실행하여 혼란도(perplexity)를 관찰하세요.

$$
𝐋(\mathbf{s})=\lVert Q(\mathbf{W}\cdot \mathbf{s})  (\mathbf{s^{-1}} \cdot \mathbf{X}) - \mathbf{W}\mathbf{X}  \rVert,  \quad\mathbf{s}= \mathbf{s_X}^{\alpha},  \mathbf{s_X} = \|X\|_1
$$
$$
\mathbf{s}^* = \text{argmin}_{\mathbf{s}} 𝐋(\mathbf{s}),\quad \alpha^*=\text{argmin}_{\alpha} 𝐋(\mathbf{s_X}^{\alpha})
$$

In [29]:
@torch.no_grad()
def scale_ln_fcs(ln, fcs, scales):
    if not isinstance(fcs, list):
        fcs = [fcs]

    scales = scales.to(ln.weight.device)

    ln.weight.div_(scales)
    if hasattr(ln, 'bias') and ln.bias is not None:
        ln.bias.div_(scales)

    for fc in fcs:
        fc.weight.mul_(scales.view(1, -1))

    for p in ln.parameters():
        assert torch.isnan(p).sum() == 0
    for fc in fcs:
        for p in fc.parameters():
            assert torch.isnan(p).sum() == 0


@torch.no_grad()
def scale_fc_fc(fc1, fc2, scales):
    assert isinstance(fc1, nn.Linear)
    assert isinstance(fc2, nn.Linear)

    scales = scales.to(fc1.weight.device)

    # fc1.weight.div_(scales.view(-1, 1))
    fc1.weight[-scales.size(0):].div_(scales.view(-1, 1))
    if fc1.bias is not None:
        fc1.bias.div_(scales.view(-1))

    fc2.weight.mul_(scales.view(1, -1))

    for p in fc1.parameters():
        assert torch.isnan(p).sum() == 0
    for p in fc2.parameters():
        assert torch.isnan(p).sum() == 0

@torch.no_grad()
def auto_scale_block(module, name, w_bit,
                     q_group_size,
                     input_feat):

    # find the best scale ratio
    def _search_module_scale(block, linears2scale: list, x, kwargs={}):

        x = x.to(next(block.parameters()).device)
        with torch.no_grad():
            org_out = block(x, **kwargs)
            if isinstance(org_out, tuple):
                org_out = org_out[0]

        s_x = x.view(-1, x.shape[-1]).abs().mean(0)
        s_x = torch.clamp(s_x, 1e-5)


        # Step 1: best_error, best_ratio, 및 best_scales를 초기화
        best_error = torch.inf
        best_ratio = -1
        best_scales = 0


        n_grid = 20
        history = []

        org_sd = {k: v.cpu() for k, v in block.state_dict().items()}
        for ratio in range(n_grid):
            # ratio is the \alpha in the formula
            ratio = ratio * 1 / n_grid

            ############### YOUR CODE STARTS HERE ###############

            # Step 2: 공식에 따라 스케일 계산
            scales = s_x ** ratio
            # scales = --- IGNORE ---

            ############### YOUR CODE ENDS HERE #################
            assert scales.shape == s_x.shape

            scales = scales / (scales.max() * scales.min()).sqrt().view(1, -1)

            for fc in linears2scale:

                scales = scales.to(fc.weight.device)

                # scale_factor를 이용해 중요한 가중치 채널의 값을 확대합니다.
                fc.weight.mul_(scales)

                fc.weight.data = pseudo_quantize_tensor(fc.weight.data, w_bit, q_group_size)

                ############### YOUR CODE STARTS HERE ###############

                # Step 3: scale_factor를 이용해 중요한 가중치 채널의 값을 다시 축소하세요.
                fc.weight.data

                ############### YOUR CODE ENDS HERE #################

            out = block(x, **kwargs)
            if isinstance(out, tuple):
                out = out[0]

            loss = (org_out - out).float().pow(2).mean().item()  # float prevents overflow
            history.append(loss)
            is_best = loss < best_error
            if is_best:
                best_error = loss
                best_ratio = ratio
                best_scales = scales
            block.load_state_dict(org_sd)

        if best_ratio == -1:
            print(history)
            raise Exception

        best_scales = best_scales.view(-1)

        assert torch.isnan(best_scales).sum() == 0, best_scales
        return best_scales.detach()

    # attention input
    inp = input_feat[name + '.self_attn.out_proj']
    inp = torch.cat([x.unsqueeze(0) for x in inp], dim=0).unsqueeze(0)
    qkv = [module.self_attn.q_proj, module.self_attn.k_proj, module.self_attn.v_proj]
    final_scales = _search_module_scale(module.self_attn, qkv, inp)
    scale_ln_fcs(module.self_attn_layer_norm, qkv, final_scales)

    # attn out
    inp = input_feat[name + '.self_attn.out_proj']
    inp = torch.cat([x.unsqueeze(0) for x in inp], dim=0)
    final_scales = _search_module_scale(module.self_attn.out_proj, [module.self_attn.out_proj], inp)
    scale_fc_fc(module.self_attn.v_proj, module.self_attn.out_proj, final_scales)

    # fc1
    inp = input_feat[name + '.fc1']
    inp = torch.cat([x.unsqueeze(0) for x in inp], dim=0)
    final_scales = _search_module_scale(module.fc1, [module.fc1], inp)
    scale_ln_fcs(module.final_layer_norm, module.fc1, final_scales)

    # fc2
    inp = input_feat[name + '.fc2']
    inp = torch.cat([x.unsqueeze(0) for x in inp], dim=0)
    final_scales = _search_module_scale(module.fc2, [module.fc2], inp)
    scale_fc_fc(module.fc1, module.fc2, final_scales)

@torch.no_grad()
def pseudo_quantize_model_weight_auto_scale(
    model, w_bit, q_group_size, input_feat
):
    from transformers.models.opt.modeling_opt import OPTDecoderLayer

    for name, module in model.named_modules():
        if isinstance(module, OPTDecoderLayer):
            auto_scale_block(module, name, w_bit, q_group_size, input_feat)

    for n, m in model.named_modules():
        if isinstance(m, nn.Linear):
            m.weight.data = pseudo_quantize_tensor(m.weight.data, n_bit=w_bit, q_group_size=q_group_size)

In [30]:
llm_model.model_reset()
pseudo_quantize_model_weight_auto_scale(llm_model.model, w_bit=3, q_group_size=128, input_feat=input_feat)
llm_model.model_changed = True

# Evaluate and delete the model
llm_model.model_evaluate(data_width=3, group_size=128)

evaluating...: 100%|██████████| 10/10 [00:00<00:00, 25.27it/s]



model perplexity: 61.74
model size: 47.12 MiB


tensor(61.7358, device='cuda:0')

# 4.2. Weight and Activation Quantization

대형 언어 모델(LLM)은 다양한 작업에서 뛰어난 성능을 보여주고 있지만, 엄청난 모델 크기로 인해 하드웨어적 장벽(메모리 크기)이 높아지고, 토큰 생성 속도가 느려집니다(메모리 대역폭). LLM의 크기와 계산량은 기하급수적으로 증가하고 있는 반면, 메모리 대역폭은 느리게 증가하고 있습니다. 이 격차는 LLM 성능에서 중요한 병목 현상입니다. 이번 실습에서는 **새로운 양자화 알고리즘(AWQ)**을 사용하여 LLM의 메모리 사용량을 줄이고 추론 속도를 가속화하는 방법을 탐구할 것입니다.

이전 수업에서는 양자화(Quantization)의 기본 방법들을 배웠습니다.

양자화에는 두 가지 유형이 있습니다:

- 가중치(weight)와 활성화(activation) 모두 양자화
    - 계산 한계가 있는 시나리오에서 더 유리합니다: 예를 들어 컨텍스트 단계나 대규모 배치 추론
    - 예시: SmoothQuant(W8A8 quantization)
- 가중치(weight)만 양자화
    - 메모리 한계가 있는 시나리오에서 더 유리합니다: 예를 들어 디코딩 단계나 단일 배치 추론.
    - 예시: AWQ(W4A16 quantization)

이 노트북에서는 OPT-125m 모델을 사용하여 SmoothQuant가 가중치와 활성화 모두에 8비트를 사용하여 FP16 모델과 동일한 정확도를 달성할 수 있음을 보여줍니다. SmoothQuant는 Linear layer에서 완전한 INT8 GEMM을 가능하게 하고, 이상값을 나타내기 위해 고정밀도 숫자를 요구하지 않습니다.

In [31]:
import torch
import torch.nn as nn

from transformers.models.opt.modeling_opt import OPTDecoderLayer
from transformers.models.bloom.modeling_bloom import BloomBlock
from transformers.models.llama.modeling_llama import LlamaDecoderLayer, LlamaRMSNorm

Uniform quantization 은 실수 값을 range$[\beta, \alpha]$에서 $[0, 2^{b} - 1]$로 매핑하는 것입니다.

Notation:

- Quantized Weight: $w_q$

- Scale factor: $s_q$

- Zero Point: $z$
\begin{equation}
s_q = \frac{\alpha - \beta}{2^{b} - 1} \tag{1},
\end{equation}
\begin{equation}
z = -\text{Round}(\beta * scale) \tag{2}
\end{equation}
\begin{equation}
w_q = \text{Clamp}(\text{Round}(\frac{w}{s_q}) + z) \tag{3},
\end{equation}



## Pseudo Quantization
아래 코드는 의사 양자화(pseudo quantization)을 위한 클래스입니다.

Pseudo Quantization는 모델의 weight와 activation을 실제로 양자화하지 않고, 양자화의 영향을 시뮬레이션하기 위해 사용됩니다. (즉, 가장 가까운 양자화된 값으로 반올림한 다음, **다시 부동 소수점으로 복원(dequantizing)**하는 것입니다.)

이 노트북에서는 실제 연산에서는 FP16을 사용하여 8비트 dynamic weight and activation qaunitzation을 시뮬레이션 합니다.

In [32]:
import torch
from torch import nn
from functools import partial

class W8A8Linear(nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        bias=True,
        act_quant="per_token",
        quantize_output=False,
        quantize_bits=8
    ):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.register_buffer(
            "weight",
            torch.randn(
                self.out_features,
                self.in_features,
                dtype=torch.float16,
                requires_grad=False,
            ),
        )
        if bias:
            self.register_buffer(
                "bias",
                torch.zeros(
                    (1, self.out_features), dtype=torch.float16, requires_grad=False
                ),
            )
        else:
            self.register_buffer("bias", None)

        if act_quant == "per_token":
            self.act_quant_name = "per_token"
            self.act_quant = partial(quantize_activation_per_token_absmax, n_bits=8)
        elif act_quant == "per_tensor":
            self.act_quant_name = "per_tensor"
            self.act_quant = partial(quantize_activation_per_tensor_absmax, n_bits=8)
        else:
            raise ValueError(f"Invalid act_quant: {act_quant}")

        if quantize_output:
            self.output_quant_name = self.act_quant_name
            self.output_quant = self.act_quant
        else:
            self.output_quant_name = "None"
            self.output_quant = lambda x: x

        self.quantize_bits = quantize_bits

    def to(self, *args, **kwargs):
        super(W8A8Linear, self).to(*args, **kwargs)
        self.weight = self.weight.to(*args, **kwargs)
        if self.bias is not None:
            self.bias = self.bias.to(*args, **kwargs)
        return self

    @torch.no_grad()
    def forward(self, x):
        q_x = self.act_quant(x)
        y = torch.functional.F.linear(q_x, self.weight, self.bias)
        q_y = self.output_quant(y)
        return q_y

    @staticmethod
    def from_float(
        module, weight_quant="per_channel", act_quant="per_token", quantize_output=False, quantize_bits=8
    ):
        assert isinstance(module, torch.nn.Linear)
        new_module = W8A8Linear(
            module.in_features,
            module.out_features,
            module.bias is not None,
            act_quant=act_quant,
            quantize_output=quantize_output,
        )
        if weight_quant == "per_channel":
            new_module.weight = quantize_weight_per_channel_absmax(
                module.weight, n_bits=new_module.quantize_bits
            )  # use 8-bit integer for weight
        elif weight_quant == "per_tensor":
            new_module.weight = quantize_weight_per_tensor_absmax(
                module.weight, n_bits=new_module.quantize_bits
            )
        else:
            raise ValueError(f"Invalid weight_quant: {weight_quant}")
        new_module.weight_quant_name = weight_quant
        if module.bias is not None:
            new_module.bias = module.bias
        return new_module

    def __repr__(self):
        return f"W8A8Linear({self.in_features}, {self.out_features}, bias={self.bias is not None}, weight_quant={self.weight_quant_name}, act_quant={self.act_quant_name}, output_quant={self.output_quant_name})"

@torch.no_grad()
def quantize_weight_per_channel_absmax(w, n_bits=8):
    # w: (out_features, in_features)
    scales = w.abs().max(dim=-1, keepdim=True)[0]
    q_max = 2 ** (n_bits - 1) - 1
    scales.clamp_(min=1e-5).div_(q_max)
    w.div_(scales).round_().mul_(scales)
    return w


@torch.no_grad()
def quantize_weight_per_tensor_absmax(w, n_bits=8):
    # w: (out_features, in_features)
    scales = w.abs().max()
    q_max = 2 ** (n_bits - 1) - 1
    scales.clamp_(min=1e-5).div_(q_max)
    w.div_(scales).round_().mul_(scales)
    return w


@torch.no_grad()
def quantize_activation_per_token_absmax(t, n_bits=8):
    t_shape = t.shape
    t.view(-1, t_shape[-1])
    scales = t.abs().max(dim=-1, keepdim=True)[0]
    q_max = 2 ** (n_bits - 1) - 1
    scales.clamp_(min=1e-5).div_(q_max)
    t.div_(scales).round_().mul_(scales)
    return t


@torch.no_grad()
def quantize_activation_per_tensor_absmax(t, n_bits=8):
    t_shape = t.shape
    t.view(-1, t_shape[-1])
    scales = t.abs().max()
    q_max = 2 ** (n_bits - 1) - 1
    scales.clamp_(min=1e-5).div_(q_max)
    t.div_(scales).round_().mul_(scales)
    return t

def quantize_opt(
    model, weight_quant="per_tensor", act_quant="per_tensor", quantize_bmm_input=True, quantize_bits=8
):
    from transformers.models.opt.modeling_opt import (
        OPTAttention,
        OPTDecoderLayer,
    )

    for name, m in model.model.named_modules():
        if isinstance(m, OPTDecoderLayer):
            m.fc1 = W8A8Linear.from_float(
                m.fc1, weight_quant=weight_quant, act_quant=act_quant, quantize_bits=8
            )
            m.fc2 = W8A8Linear.from_float(
                m.fc2, weight_quant=weight_quant, act_quant=act_quant, quantize_bits=8
            )
        elif isinstance(m, OPTAttention):
            # Her we simulate quantizing BMM inputs by quantizing the output of q_proj, k_proj, v_proj
            m.q_proj = W8A8Linear.from_float(
                m.q_proj,
                weight_quant=weight_quant,
                act_quant=act_quant,
                quantize_output=quantize_bmm_input, quantize_bits=8
            )
            m.k_proj = W8A8Linear.from_float(
                m.k_proj,
                weight_quant=weight_quant,
                act_quant=act_quant,
                quantize_output=quantize_bmm_input, quantize_bits=8
            )
            m.v_proj = W8A8Linear.from_float(
                m.v_proj,
                weight_quant=weight_quant,
                act_quant=act_quant,
                quantize_output=quantize_bmm_input, quantize_bits=8
            )
            m.out_proj = W8A8Linear.from_float(
                m.out_proj, weight_quant=weight_quant, act_quant=act_quant, quantize_bits=8
            )
    return model

이제 quantized 8-bit 모델의 혼란도(perplexity)와 크기를 평가해 봅시다.

In [33]:
llm_model.model_reset()
model_w8a8 = quantize_opt(llm_model.model, quantize_bits=8)
print(model_w8a8)
llm_model.model_change(model_w8a8)

# Evaluate and delete the model
llm_model.model_evaluate(data_width=8, group_size=128)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=

evaluating...: 100%|██████████| 10/10 [00:01<00:00,  9.52it/s]


model perplexity: 31.64
model size: 121.77 MiB


tensor(31.6428, device='cuda:0')

모델 크기가 줄어든 것은 확인할 수 있지만, 혼란도(perplexity)는 약간 증가했습니다.

AWQ의 관찰에서와 마찬가지로, LLM의 활성화(activations)에서 일부 채널에 **아웃라이어(outliers)**가 소량 발생하고 있습니다. 특정 채널에 아웃라이어가 있는 경우, 이는 **모든 토큰에서 지속적으로 나타납니다.**

주어진 토큰에 대한 채널 간의 분산(variance)은 크지만(일부 채널의 활성화는 매우 크고, 대부분은 작습니다), 특정 채널의 크기(magnitude)가 토큰 간에 가지는 분산은 작습니다(아웃라이어 채널은 지속적으로 큽니다).

Smoothquant 논문의 관찰에 따르면, 이러한 현상은 activation에서만 발견되는 현상이며, weight에서는 발견되지 않습니다.

그렇기 떄문에, AWQ 기법과 같이 weight에 대해서는 4bit 정도의 낮은 정밀도로 quantization이 가능하지만, activation에서는 매우 큰 정확도 하락이 발생합니다.


## Migrate the quantization difficulty from activations to weights

양자화 오류(quantization error)를 줄이기 위해서는 모든 채널에 대해 유효 양자화 비트수를 증가시켜야 합니다.

그러나 연산 과정에서 activation은 채널 차원이 아닌 토큰 차원에서 행렬 곱셈이 이루어지기 때문에, per-channel quantization을 도입하는 것으로는 속도의 향상을 불러올 수 없습니다.

대신 Smoothquant에서는 activation을 per-channel smoothing fator $\mathbf{s}$로 나누어 "smooth"하는 방법을 제안합니다.

이 방법은 각 activation 채널에 독립적인 스케일링 인자를 적용하여 입력 활성화의 이상치(outliers)를 평활화하고, 이로 인해 양자화 과정에서 발생할 수 있는 오류와 정확도 손실을 최소화합니다.

즉, 각 활성화 채널의 스케일을 조정함으로써 전체 행렬의 양자화가 더욱 효과적이고 안정적으로 이루어질 수 있도록 합니다.

$$
Y = (X \text{diag}(s)^{-1}) \cdot (\text{diag}(s)W) = \hat{X}\hat{W}
$$


여기서 입력 𝑋는 일반적으로 이전의 선형 연산(예: Linear layer, Layer Normalization 등)에서 생성되므로, 우리는 스케일링 팩터 $s$를 이전 레이어의 파라미터에 오프라인으로 미리 결합할 수 있습니다. 이렇게 하면 추가적인 스케일링으로 인한 커널 호출 오버헤드가 발생하지 않습니다.

# [실습 3] Quantization difficulty migration

실습을 통해 weight에는 s를 곱하고, activation에는 s를 나누어 quantization 난이도를 분배해 준 다음, quantization을 진행해 혼란도(perplexity)의 변화를 관찰해보세요.

일반적인 Transformer의 레이어 구조는 다음과 같습니다:

1. Self-Attention Block

    Input → LayerNorm → Query/Key/Value 생성 (FC) → Attention 연산 → Softmax → Attention Output

2. Feed-Forward Network (FFN)

    Attention Output → LayerNorm → FC1 → 활성화 함수 (ReLU, GELU 등) → FC2 → Output

그러므로 Transformer의 레이어에서 LayerNorm의 가중치에 스케일링 팩터 $s$를 나누고, FC의 가중치에 스케일링 팩터 $s$를 곱하면 weight에는 s를 곱하고, activation에는 s를 나누는 효과를 얻을 수 있습니다.

아래 코드를 수정하여 LayerNorm의 가중치에 스케일링 팩터 $s$를 나누고, FC의 가중치에 스케일링 팩터 $s$를 곱해주세요.

In [40]:
@torch.no_grad()
def smooth_lm_by_scale(model, scale):
    for name, module in model.named_modules():
        if isinstance(module, OPTDecoderLayer):
            attn_ln = module.self_attn_layer_norm
            qkv = [
                module.self_attn.q_proj,
                module.self_attn.k_proj,
                module.self_attn.v_proj,
            ]
            smooth_ln_fcs_by_scale(attn_ln, qkv, scale)

            ffn_ln = module.final_layer_norm
            fc1 = module.fc1
            smooth_ln_fcs_by_scale(ffn_ln, fc1, scale)

@torch.no_grad()
def smooth_ln_fcs_by_scale(ln, fcs, scale):
    if not isinstance(fcs, list):
        fcs = [fcs]
    assert isinstance(ln, nn.LayerNorm)
    for fc in fcs:
        assert isinstance(fc, nn.Linear)
    ############### YOUR CODE STARTS HERE ###############
    # Step 1: layernorm의 weight와 bias를 scale로 나누어주세요. (hint: div_()함수를 통해 tensor 전체를 특정한 값으로 나누어 줄 수 있습니다.)
    ln.weight.div_(scale)
    ln.bias.div_(scale)
    ############### YOUR CODE ENDS HERE #################

    for fc in fcs:
        ############### YOUR CODE STARTS HERE ###############
        # Step 2: fc의 weight에 scale을 곱해주세요. (hint: mul_()함수를 통해 tensor 전체에 특정한 값을 곱해 줄 수 있습니다.)
        fc.weight.mul_(scale)
        ############### YOUR CODE ENDS HERE #################


In [41]:
llm_model.model_reset()
smooth_lm_by_scale(llm_model.model, 5)
model_smoothquant_scale = quantize_opt(llm_model.model)
print(model_smoothquant_scale)
llm_model.model_change(model_smoothquant_scale)

# Evaluate the model
llm_model.model_evaluate(data_width=8, group_size=128)

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (v_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (q_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=per_tensor)
            (out_proj): W8A8Linear(768, 768, bias=True, weight_quant=per_tensor, act_quant=per_tensor, output_quant=None)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=

evaluating...: 100%|██████████| 10/10 [00:00<00:00, 10.32it/s]


model perplexity: 31.65
model size: 121.77 MiB


tensor(31.6455, device='cuda:0')

스케일링을 통해서 가중치와 활성화의 양자화 난이도를 적절히 분배하고, 가중치와 활성화를 모두 8bit로 유지할 수 있었습니다.

이번에는 코드에서 서로 다른 다양한 스케일링 팩터 $s$(예: 0.001, 0.01, 1)을 시도하고 혼란도(perplexity)의 변화를 관찰해보겠습니다.

In [42]:
for scale_factor in [0.001,0.01,0.1]:
    llm_model.model_reset()
    smooth_lm_by_scale(llm_model.model,scale_factor)
    model = quantize_opt(llm_model.model)
    llm_model.model_change(model)

    # Evaluate the model
    print(f"scale_factor={scale_factor}")
    llm_model.model_evaluate(data_width=8, group_size=128)

scale_factor=0.001


evaluating...: 100%|██████████| 10/10 [00:01<00:00,  9.92it/s]



model perplexity: 31.71
model size: 121.77 MiB
scale_factor=0.01


evaluating...: 100%|██████████| 10/10 [00:01<00:00,  9.86it/s]



model perplexity: 31.62
model size: 121.77 MiB


HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 8s [Retry 4/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 8s [Retry 5/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json


HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://huggingface.co/api/models/facebook/opt-125m

## [실습 4] Scale factor sampling

스케일링 팩터 $s$값의 설정에 따라 혼란도(perplexity)가 먼저 변화하는 것을 관찰했나요?


우리의 목표는 각 채널별 스케일링 팩터 s를 선택하여  X̂ = Xdiag(s)⁻¹가 양자화하기 쉽도록 만드는 것입니다.

양자화 오류를 줄이기 위해 모든 채널의 유효 양자화 비트를 늘려야 합니다.

가장 간단한 선택은 채널별로 서로 다른 스케일링 팩터를 설정하는 것입니다.

스케일링 팩터를 weight의 최대값으로 설정하면 weight의 양자화 난이도가 쉬워집니다. 그러나 activation의 양자화는 어려워집니다.

반대로, 스케일링 팩터를 activation의 최대값으로 설정하면 weight의 양자화가 어려워집니다.

우리는 weight와 activation의 양자화 난이도 사이에서 균형을 맞추기 위해 스케일링 팩터 s를 다음과 같이 설정합니다.

$s = \max(|X|)^{\alpha} / \max(|W|)^{1-\alpha}$

아래 코드를 수정하여 스케일링 팩터 s를 위의 식과 같이 설정해주세요.

In [43]:
@torch.no_grad()
def smooth_ln_fcs(ln, fcs, act_scales, alpha=0.5):
    if not isinstance(fcs, list):
        fcs = [fcs]
    assert isinstance(ln, nn.LayerNorm)
    for fc in fcs:
        assert isinstance(fc, nn.Linear)
        assert ln.weight.numel() == fc.in_features == act_scales.numel()

    device, dtype = fcs[0].weight.device, fcs[0].weight.dtype
    act_scales = act_scales.to(device=device, dtype=dtype)
    weight_scales = torch.cat(
        [fc.weight.abs().max(dim=0, keepdim=True)[0] for fc in fcs], dim=0
    )
    weight_scales = weight_scales.max(dim=0)[0].clamp(min=1e-5)

    scales = (
        ############### YOUR CODE STARTS HERE ###############
        #Activation Scales 값과 Weight Scales 값에 alpha를 적절히 거듭제곱해주어야 합니다.
        #Hint: pow()함수를 통해서 거듭제곱을 사용할 수 있습니다.
        (act_scales ** alpha) / (weight_scales ** (1 - alpha))

        ############### YOUR CODE ENDS HERE #################
    )

    scales.clamp(min=1e-5).to(device).to(dtype)

    ln.weight.div_(scales)
    ln.bias.div_(scales)

    for fc in fcs:
        fc.weight.mul_(scales.view(1, -1))


여기서 활성화 범위는 동적이며 입력 샘플에 따라 달라집니다.

사전 훈련 데이터 세트의 보정 샘플을 사용하여 활성화 채널의 크기를 추정해보겠습니다.

아래 코드를 실행하면 512개의 사전 훈련 샘플 데이터 세트를 통해 자동으로 적절한 스케일링 팩터 $s$값을 찾아 양자화를 진행합니다.

In [44]:
import functools
def get_act_scales(model, tokenizer, dataset_path, num_samples=512, seq_len=512):
    model.eval()
    device = next(model.parameters()).device
    act_scales = {}

    def stat_tensor(name, tensor):
        hidden_dim = tensor.shape[-1]
        tensor = tensor.view(-1, hidden_dim).abs().detach()
        comming_max = torch.max(tensor, dim=0)[0].float().cpu()
        if name in act_scales:
            act_scales[name] = torch.max(act_scales[name], comming_max)
        else:
            act_scales[name] = comming_max

    def stat_input_hook(m, x, y, name):
        if isinstance(x, tuple):
            x = x[0]
        stat_tensor(name, x)

    hooks = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Linear):
            hooks.append(
                m.register_forward_hook(functools.partial(stat_input_hook, name=name))
            )

    dataset = load_dataset("json", data_files=dataset_path, split="train")
    dataset = dataset.shuffle(seed=42)

    for i in tqdm(range(num_samples)):
        input_ids = tokenizer(
            dataset[i]["text"], return_tensors="pt", max_length=seq_len, truncation=True
        ).input_ids.to(device)
        model(input_ids)

    for h in hooks:
        h.remove()

    return act_scales

In [45]:
@torch.no_grad()
def smooth_lm(model, scales, alpha=0.5):
    for name, module in model.named_modules():
        if isinstance(module, OPTDecoderLayer):
            attn_ln = module.self_attn_layer_norm
            qkv = [
                module.self_attn.q_proj,
                module.self_attn.k_proj,
                module.self_attn.v_proj,
            ]
            qkv_input_scales = scales[name + ".self_attn.q_proj"]
            smooth_ln_fcs(attn_ln, qkv, qkv_input_scales, alpha)

            ffn_ln = module.final_layer_norm
            fc1 = module.fc1
            fc1_input_scales = scales[name + ".fc1"]
            smooth_ln_fcs(ffn_ln, fc1, fc1_input_scales, alpha)

In [ ]:
# model_path = "facebook/opt-125m"
# llm_model = LLMModel(model_path)

HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 1s [Retry 1/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 2s [Retry 2/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 4s [Retry 3/5].
HTTP Error 429 thrown while requesting HEAD https://huggingface.co/facebook/opt-125m/resolve/main/config.json
Retrying in 8s [Retry 4/5].


In [ ]:
llm_model.model_reset()

act_scales = get_act_scales(
        llm_model.model, llm_model.tokenizer, datapath, 512, 512)
smooth_lm(llm_model.model, act_scales)
model_sampled = quantize_opt(llm_model.model)
llm_model.model_change(model_sampled)

# Evaluate the model
llm_model.model_evaluate(data_width=8, group_size=128)

del llm_model
torch.cuda.empty_cache()

Generating train split: 0 examples [00:00, ? examples/s]

# Rotation Based Quantization
최근에는 LLM Quantization의 Outlier 문제를 해결하기 위해 Rotation Matrix를 곱해서 Outlier를 제거하는 방법들이 제시되고 있습니다. (QuaRot, SpinQuant 등)

![SpinQuant](https://raw.githubusercontent.com/facebookresearch/SpinQuant/refs/heads/main/SpinQuant.png)

해당 기법들은 직교 행렬의 아래와 같은 특성을 활용합니다.

1. $$ R\,R^{T} = I $$

2. 벡터 V에 직교 행렬 R을 곱하면, 길이는 같으나 방향이 바뀐 벡터 V'를 구할 수 있음



## 수식 유도

1. 원래 연산  
   $$
     y = x\,W
   $$

2. 직교 행렬 $R$ 도입  
   $$
     y = x\,I\,W = x\,(RR^{T})\,W = (x\,R)\,(R^{T}\,W)
   $$

3. 새로운 변수로 정의
   $$
     x' = x\,R^{T}
     \quad
     W' = R\,W
   $$
   이때  
   $$
     y = x'\,W'
     = x\,W
   $$  
   가 수식적으로 보존됨
  
## 장점
  - 회전을 통해 분포를 분산시켜 Outlier 현상을 완화  
  - 사전에 Weight에 곱해두는 것을 통해 적용 가능 및 온라인 연산 제거 가능

In [ ]:
llm_model = LLMModel("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [ ]:
def get_orthogonal_matrix(size, mode="random", dtype=torch.float32, device="cpu"):
    if mode == "random":
        A = torch.randn(size, size, dtype=torch.float32, device=device)
        Q, R = torch.linalg.qr(A)
        Q *= torch.sign(torch.diag(R)).unsqueeze(0)
        return Q.to(dtype)
    else:
        raise ValueError(f"Unknown mode: {mode}")

dummy_vector = torch.randn(100)
dummy_vector[50] = 30

rotation_matrix = get_orthogonal_matrix(100)
rotated_vector = rotation_matrix @ dummy_vector

def plot_vector(vec, title):
    plt.figure(figsize=(5, 2))
    plt.plot(vec.abs().numpy(), linewidth=2)
    plt.title(title, fontsize=14)
    plt.xlabel('Index', fontsize=12)
    plt.ylabel('Absolute Value', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

print(rotation_matrix)
print(torch.round(rotation_matrix @ rotation_matrix.T))
plot_vector(dummy_vector, 'Original Vector (Absolute Values)')
plot_vector(rotated_vector, 'Rotated Vector (Absolute Values)')

In [ ]:
def quantize_tinyllama(
    model, weight_quant="per_tensor", act_quant="per_tensor", quantize_bmm_input=True, quantize_bits=4
):
    """
    Quantize TinyLlama model using W8A8Linear layers.
    """
    for name, m in model.model.named_modules():
        if isinstance(m, torch.nn.Linear):
            # Skip lm_head as it's handled separately
            if "lm_head" in name:
                continue

            # Create quantized linear layer
            quantized_layer = W8A8Linear.from_float(
                m, weight_quant=weight_quant, act_quant=act_quant, quantize_bits=4
            )

            # Replace the original layer with quantized one
            # We need to find the parent module and replace the child
            parent_name = ".".join(name.split(".")[:-1])
            child_name = name.split(".")[-1]

            if parent_name:
                parent = model.model.get_submodule(parent_name)
                setattr(parent, child_name, quantized_layer)
            else:
                # Root level module
                setattr(model.model, child_name, quantized_layer)

    return model

## Layernorm ↔ Linear Fusion

실제 모델에서는 Rotation Matrix 사이에 Layernorm이 존재하는 경우가 많습니다.

![LayerNorm](https://miro.medium.com/v2/resize:fit:1252/1*kC-cWBWDEZpkSCtYIUsj4w.png)

Normalization Layer의 영향으로 Rotation Matrix가 곱해지지 못해 정상적으로 제거되지 못하고 모델의 연산이 부정확해지는 문제가 발생할 수 있습니다.

따라서, Rotation 적용 이전에 Normalization Layer를 Linear Layer와 Fusion해야 합니다.

In [ ]:
# Layer Norm Fusion Functions for TinyLlama
def fuse_ln_linear(layernorm, linear_layers):
    """
    Fuse the linear operations in Layernorm into the adjacent linear blocks.
    """
    for linear in linear_layers:
        linear_dtype = linear.weight.dtype

        # Calculating new weight and bias
        W_ = linear.weight.data.double()
        linear.weight.data = (W_ * layernorm.weight.double()).to(linear_dtype)

        if hasattr(layernorm, "bias") and layernorm.bias is not None:
            if linear.bias is None:
                linear.bias = torch.nn.Parameter(
                    torch.zeros(linear.out_features, dtype=torch.float64)
                )
            linear.bias.data = linear.bias.data.double() + torch.matmul(
                W_, layernorm.bias.double()
            )
            linear.bias.data = linear.bias.data.to(linear_dtype)

def fuse_layer_norms_tinyllama(model):
    """
    Fuse layer norms for TinyLlama model structure.
    """
    # Embedding fusion
    for W in [model.model.embed_tokens]:
        W_ = W.weight.data.double()
        W.weight.data = (W_ - W_.mean(dim=-1, keepdim=True)).to(W.weight.data.dtype)

    layers = [layer for layer in model.model.layers]

    # Fuse the linear operations in Layernorm into the adjacent linear blocks.
    for layer in layers:
        # fuse the input layernorms into the linear layers
        fuse_ln_linear(
            layer.input_layernorm,
            [layer.self_attn.q_proj, layer.self_attn.k_proj, layer.self_attn.v_proj]
        )
        fuse_ln_linear(
            layer.post_attention_layernorm,
            [layer.mlp.gate_proj, layer.mlp.up_proj]
        )

        # Set layernorm weights to ones
        W_norm = layer.input_layernorm.weight.data
        layer.input_layernorm.weight.data = torch.ones_like(W_norm)
        W_norm = layer.post_attention_layernorm.weight.data
        layer.post_attention_layernorm.weight.data = torch.ones_like(W_norm)

    # Fuse final norm into lm_head
    fuse_ln_linear(
        model.model.norm,
        [model.lm_head],
    )
    W_norm = model.model.norm.weight.data
    model.model.norm.weight.data = torch.ones_like(W_norm)

## [실습 5] Rotate Matrix 적용

QuaRot는 R1만 사용하여 Roation을 적용합니다.

그를 보완한 SpinQuant에서는 R2, R3, R4 등 다양한 Rotation Matrix를 적용하여 Quantization 정확도를 높입니다.

그러나, R2는 R1과 유사하게 적용 가능하고, R3와 R4는 On-line에서 구해지는 Matrix이기 때문에 구현 난이도를 낮추기 위해 QuaRot을 구현하는 것으로 하겠습니다.

QuaRot 혹은 SpinQuant 그림을 참고하셔서 각각의 연산에 R1이 어떻게 적용될 것인지 구현해보시기 바랍니다.

In [ ]:
def rotate_model_weight(
    model, R1
):
    for n, m in model.named_modules():
      ############### YOUR CODE STARTS HERE ###############
      # Pytorch에서 @ 연산이 Dot Product 임을 사용하시기 바랍니다.
      # nn.Linear 연산의 Parameter는 W^T 형태로 저장되어 있다는 것을 유의하시기 바랍니다.
      # Embedding Parameter Shape : (Num_Tokens, Hidden_dim)
      # Linear Parameter Shape : (Output_Channel, Input_Channel)
      # Roation Matrix Shape : (Hidden_dim, Hidden_dim)

      if isinstance(m, nn.Embedding):
        W_ = m.weight.data
        m.weight.data =

      if isinstance(m, nn.Linear):
        if "o_proj" in n or "down_proj" in n:
          # Att Out Proj, FFN Down Proj
          W_ = m.weight.data
          m.weight.data =

        else:
          # QKV Proj, FFN Up Proj, FFN Gate Proj
          W_ = m.weight.data
          m.weight.data =

      ############### YOUR CODE ENDS HERE #################

      torch.cuda.empty_cache()

In [ ]:
print("\nOriginal 모델 성능 측정 중...")
llm_model.model_reset()
original_perplexity = llm_model.model_evaluate(data_width=16, group_size=128)
output_orig = llm_model.model(llm_model.testenc[:,:500].to(llm_model.model.device), output_hidden_states=True)
torch.cuda.empty_cache()

In [ ]:
Q_BITS = 8

print("\nQuantization만 적용한 모델 성능 측정 중...")
llm_model.model_reset()
model = quantize_tinyllama(llm_model.model, quantize_bits=Q_BITS)
llm_model.model_change(model)
quantized_only_perplexity = llm_model.model_evaluate(data_width=Q_BITS, group_size=128)
torch.cuda.empty_cache()

In [ ]:
print("\nRotation + Quantization 적용한 모델 성능 측정 중...")
llm_model.model_reset()
fuse_layer_norms_tinyllama(llm_model.model)
hidden_size = llm_model.model.config.hidden_size
R1_random = get_orthogonal_matrix(hidden_size, mode="random", dtype=llm_model.model.dtype, device=llm_model.model.device)
rotate_model_weight(llm_model.model, R1_random)
model = quantize_tinyllama(llm_model.model, quantize_bits=Q_BITS)
llm_model.model_change(model)
rotation_quantized_perplexity = llm_model.model_evaluate(data_width=Q_BITS, group_size=128)
output_rotated = llm_model.model(llm_model.testenc[:,:500].to(llm_model.model.device), output_hidden_states=True)
torch.cuda.empty_cache()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_2d(data, title):
    # Get dimensions of hidden states - handle different shapes
    if len(data.shape) == 3:  # (batch, seq_len, hidden_dim)
        data = data[0]  # Take first batch
    elif len(data.shape) == 2:  # (seq_len, hidden_dim)
        pass
    else:
        print(f"Unexpected data shape: {data.shape}")
        return

    seq_len, hidden_dim = data.shape

    mean_activations = np.mean(np.abs(data), axis=0)

    # 2D Plot
    plt.figure(figsize=(12, 6))
    plt.plot(np.arange(hidden_dim), mean_activations, color='blue', linewidth=1.5)

    plt.xlabel("Hidden Dimension", fontsize=12)
    plt.ylabel("Absolute Value", fontsize=12)
    plt.title(f"Hidden States Mean Activation - {title}", fontsize=14)

    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

# 사용 예시
plot_2d(output_orig.hidden_states[16][:, 100:].detach().cpu().numpy(), "Original")
plot_2d(output_rotated.hidden_states[16][:, 100:].detach().cpu().numpy(), "Rotated")